In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
df = pd.read_csv("amz_uk_price_prediction_dataset.csv")
df = df[df.columns[df.isnull().mean() < 0.8]]
df.head()

In [ ]:
#Part 1: Analyzing Best-Seller Trends Across Product Categories
#Objective: Understand the relationship between product categories and their best-seller status.

#Crosstab Analysis:
#Create a crosstab between the product category and the isBestSeller status.
#Are there categories where being a best-seller is more prevalent?
#Hint: one option is to calculate the proportion of best-sellers for each category and then sort the categories based on this proportion in descending order.

#Statistical Tests:
#Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.
#Compute Cramér's V to understand the strength of association between best-seller status and category.

#Visualizations:
#Visualize the relationship between product categories and the best-seller status using a stacked bar chart.

In [ ]:
crosstab = pd.crosstab(df['category'], df['isBestSeller'])
crosstab['Proportion'] = crosstab[True] / (crosstab[True] + crosstab[False])
# Sort categories by proportion in descending order
sorted_by_proportion = crosstab.sort_values(by='Proportion', ascending=False)
sorted_by_proportion

In [ ]:
#Statistical Tests:
from scipy.stats import chi2_contingency
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab)
chi2_statistic, chi2_p_value

In [ ]:
from scipy.stats.contingency import association

# Computing the association between variables in 'crosstab' using the "cramer" method
association(crosstab, method="cramer")

In [ ]:
#Visualizations:
sorted_by_proportion.head(20).plot(kind="bar", stacked=True);

In [ ]:
#Part 2: Exploring Product Prices and Ratings Across Categories and Brands
#Objective: Investigate how different product categories influence product prices.

#Preliminary Step: Remove outliers in product prices.
#For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.
#Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.

#Violin Plots:
#Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.
#Which product category tends to have the highest median price? Don't filter here by top categories.                                                                                                               
                                                                                                                                                                                                                                                                                          
#Bar Charts:
#Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
#Which product category commands the highest average price? Don't filter here by top categories.
                                                                                                                                             
#Box Plots:
#Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
#Which category tends to receive the highest median rating from customers? Don't filter here by top categories

In [ ]:
#Preliminary Step: Remove outliers in product prices.
def tukeys_test_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify the outliers
    outliers = data[(data < lower_bound) | (data > upper_bound)]

    return outliers

In [ ]:
data_series = df['price'] 
outliers = tukeys_test_outliers(data_series)
print(outliers)

In [ ]:
#Violin Plots:
#Use a violin plot to visualize the distribution of price across different product categories. 
#Filter out the top 20 categories based on count for better visualization.
#Which product category tends to have the highest median price? Don't filter here by top categories.   
# Find the top categories by count
# Find the top categories by count
top_categories = df['category'].value_counts().nlargest(20).index

# Filter the DataFrame for these top categories
df_top_categories = df[df['category'].isin(top_categories)]
plt.figure(figsize=(12, 8))
sns.violinplot(data=df_top_categories, x='category', y='price')
plt.xticks(rotation=45, ha='right')
plt.title('Price Distribution Across Top 20 Product Categories')
plt.xlabel('Product Category')
plt.ylabel('Price')
plt.tight_layout()
plt.show()


In [ ]:
category_medians = df.groupby('category')['price'].median()
print("Product category is:" ,category_medians.idxmax())
print("The highest median price is:", category_medians.max())

In [ ]:
#Bar Charts:
#Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
#Which product category commands the highest average price? Don't filter here by top categories.

category_avg_price = df.groupby('category')['price'].mean().reset_index()
top_10_categories = category_avg_price.sort_values('price', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_10_categories, x='category', y='price', hue ='category', legend=False,palette='coolwarm')
plt.xticks(rotation=90)

plt.show()

In [ ]:
cat_highest_average = df.groupby('category')['price'].mean()
print("Category name is:" ,cat_highest_average.idxmax())
print("The highest averag price is:", cat_highest_average.max())

In [ ]:
#Box Plots:
#Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
#Which category tends to receive the highest median rating from customers? Don't filter here by top categories
top_10_categories = df['category'].value_counts().nlargest(10).index

# 2. Filter DataFrame to include only these top categories
df_top_10 = df[df['category'].isin(top_10_categories)]

# 3. Create box plots for these top categories
plt.figure(figsize=(14, 8))
sns.boxplot(data=df_top_10, x='category', y='stars', palette='coolwarm')
plt.xticks(rotation=45, ha='right')
plt.title('Product Rating Distribution Across Top 10 Categories')
plt.xlabel('Product Category')
plt.ylabel('Ratings')
plt.tight_layout()
plt.show()


In [ ]:
median_ratings = df.groupby('category')['stars'].median()
print("category with the highest median rating is:" , median_ratings.idxmax())
print("the highest median rating is:",median_ratings.max())

In [ ]:
#Part 3: Investigating the Interplay Between Product Prices and Ratings
#Objective: Analyze how product ratings (stars) correlate with product prices.

#Correlation Coefficients:
#Calculate the correlation coefficient between price and stars.
#Is there a significant correlation between product price and its rating?

#Visualizations:
#Use a scatter plot to visualize the relationship between product rating and price. What patterns can you observe?
#Use a correlation heatmap to visualize correlations between all numerical variables.
#Examine if product prices typically follow a normal distribution using a QQ plot.


In [ ]:
correlation = df['price'].corr(df['stars'])
correlation

In [ ]:
sns.scatterplot(data=df, x='stars', y='price');

In [ ]:
corr_matrix = df.select_dtypes("number").corr(method = "spearman")
corr_matrix

In [ ]:
sns.heatmap(corr_matrix, annot = True);

In [ ]:
import statsmodels.api as sm

sm.qqplot(df['price'], line='s');